# **Formulación Matemática de Autoencoders**

El estudio de los autoencoders, en general, es importante para poder interpretar lo que aprenden los modelos Transformer. Esta es una de las herramientas más 
utilizadas actualmente para la interpretación de dichos modelos. En esta sección se analizará la estructura matemática de los autoencoders simples y, posteriormente,
 de los autoencoders sparsos, los cuales incorporan pequeñas penalizaciones en la función de pérdida. Estas penalizaciones les otorgan propiedades particulares que 
 pueden ser útiles para nuestros fines.


## **Autoencoder Simple**


### **Definición del Autoencoder**
Un **autoencoder** es una función compuesta $ h: \mathcal{X} \to \mathcal{X} $ definida por la composición de dos funciones diferenciables:

$$
h(\mathbf{x}) = g_{\phi}\bigl(f_{\theta}(\mathbf{x})\bigr)
$$


donde, $f_{\theta}: \mathcal{X} \to \mathbb{R}^m$ es la **función de codificación (encoder)** y $g_{\phi}: \mathbb{R}^m \to \mathcal{X}$ es la **función de decodificación (decoder)**.

El objetivo del autoencoder es encontrar los parámetros $\theta$ y $\phi$ tales que $h(\mathbf{x}) \approx \mathbf{x}$, minimizando una función de pérdida adecuada.


### **Codificador (Encoder)**
El encoder transforma la entrada $\mathbf{x} \in \mathbb{R}^n$ en una representación latente $\mathbf{z} \in \mathbb{R}^m$, con $m < n$ en el caso de reducción de dimensionalidad:

$$
\mathbf{z} = f_{\theta}(\mathbf{x}) = \sigma\bigl(W_e \,\mathbf{x} + \mathbf{b}_e\bigr)
$$

donde, $W_e \in \mathbb{R}^{m \times n}$ es la **matriz de pesos del encoder**, $\mathbf{b}_e \in \mathbb{R}^{m}$ es el **vector de sesgo**, $\sigma: \mathbb{R} \to \mathbb{R}$ es una función de activación (**ReLU**, **Sigmoid**, **Tanh**) y $\mathbf{z} \in \mathbb{R}^{m}$ es la representación latente.


### **Decodificador (Decoder)**
El decoder reconstruye la entrada original a partir de $\mathbf{z}$:

$$
\hat{\mathbf{x}} = g_{\phi}(\mathbf{z}) = \sigma'\bigl(W_d \,\mathbf{z} + \mathbf{b}_d\bigr)
$$

donde, $W_d \in \mathbb{R}^{n \times m}$ es la **matriz de pesos del decoder**, $\mathbf{b}_d \in \mathbb{R}^{n}$ es el **vector de sesgo**, $\sigma': \mathbb{R} \to \mathbb{R}$ es una función de activación (puede diferir de $\sigma$) y $\hat{\mathbf{x}} \in \mathbb{R}^n$ es la **reconstrucción de la entrada**.


### **Función de Pérdida**

Es importante considerar que la función de pérdida puede variar. En principio, el error cuadrático medio (Mean Squared Error, MSE) es una de las más utilizadas en autoencoders simples; sin embargo, dependiendo de la tarea a realizar, en algunos casos conviene más utilizar una u otra.

Para un conjunto de datos 

$$
\mathcal{D} = \{\mathbf{x}_i\}_{i=1}^{N},
$$

el entrenamiento del autoencoder minimiza la diferencia entre la entrada $\mathbf{x}_i$ y la reconstrucción $\hat{\mathbf{x}}_i$. Usamos el **Error Cuadrático Medio (MSE)** promediado:

$$
\mathcal{L}_{MSE} = \frac{1}{N} \sum_{i=1}^{N}
\left(
    \frac{1}{n} \sum_{j=1}^{n} 
    \bigl(x_{i,j} - \hat{x}_{i,j}\bigr)^2
\right).
$$

donde, $N$ es el número total de muestras, $d$ es la dimensión de cada muestra $\mathbf{x}_i$, $x_{i,j}$ y $\hat{x}_{i,j}$ representan la $j$-ésima componente de la muestra $\mathbf{x}_i$ y de su reconstrucción, respectivamente.

<br>
<details>
<summary>Nota: Sobre algunas otras funciones de pérdida. </summary>

El **Error cuadrático medio** es ideal para datos continuos, como imágenes con valores reales. Es fácil de usar y da resultados estables, pero puede generar salidas borrosas porque penaliza fuertemente los errores grandes.

**Binary Crossentropy** se usa cuando los datos están entre 0 y 1, como imágenes normalizadas. Funciona bien con activaciones como sigmoide, y modela la probabilidad de cada píxel o bit.

La Binary Crossentropy se expresa como:

$$
\mathcal{L}_{BCE} = 
- \frac{1}{N \times d} \sum_{i=1}^{N} \sum_{j=1}^{d}
\Bigl[
    x_{i,j} \, \log\bigl(\hat{x}_{i,j}\bigr) 
    +
    \bigl(1 - x_{i,j}\bigr) \, \log\bigl(1 - \hat{x}_{i,j}\bigr)
\Bigr],
$$

donde $ d $ es la dimensión de cada muestra, $ x_{i,j} \in \{0,1\} $ y $ \hat{x}_{i,j} $ es la probabilidad estimada por el modelo para dicha componente. Esta función de pérdida castiga fuertemente aquellas predicciones en las que $ \hat{x}_{i,j} $ difiere de $ x_{i,j} $ con alto grado de confianza (debido al uso del logaritmo).


**Categorical Crossentropy** es más adecuada cuando la salida son categorías, como texto o etiquetas. 
Para cuando cada muestra $ \mathbf{x}_i $ pertenece a una de $ K $ categorías y se representa en formato *one-hot* (solo una de sus $ K $ posiciones es 1, mientras que el resto son 0), tenemos la **Categorical Crossentropy**:

$$
\mathcal{L}_{CCE} =
- \frac{1}{N}
\sum_{i=1}^{N}
\sum_{k=1}^{K}
x_{i,k} \,\log\bigl(\hat{x}_{i,k}\bigr),
$$

donde $ x_{i,k} $ es 1 si la clase $ k $ es la correcta para la muestra $ \mathbf{x}_i $, y $ \hat{x}_{i,k} $ es la probabilidad que el modelo asigna a la clase $ k $. El objetivo en este caso es alinear la distribución pronosticada con la verdadera, penalizando fuertemente cuando la probabilidad de la clase correcta resulta ser baja.


</details>


### **Optimización**
El objetivo es encontrar los parámetros $\theta$ y $\phi$ que minimicen la función de pérdida:

$$
\theta^*, \phi^* = \arg \min_{\theta, \phi} \,\mathcal{L}_{MSE}.
$$

La optimización se resuelve mediante **descenso de gradiente**, por ejemplo usando una tasa de aprendizaje $\eta$:

$$
\theta \leftarrow \theta - \eta\,\nabla_{\theta} \,\mathcal{L}_{MSE},
\quad
\phi \leftarrow \phi - \eta\,\nabla_{\phi} \,\mathcal{L}_{MSE}.
$$


<details>
<summary>Nota: Sobre la optimizacion</summary>

Hay diferentes formas de optimizacion. 
</details>


<br>

## **Sparse Autoencoder**

Los sparse autoencoders son útiles para interpretar modelos Transformer porque permiten identificar representaciones latentes significativas y más fácilmente interpretables. Al forzar que solo una pequeña parte del espacio latente se active para cada ejemplo, el modelo tiende a representar conceptos más específicos y dispersos. Esto facilita analizar qué tipo de información se activa internamente en respuesta a una entrada, ayudando a entender mejor cómo el Transformer organiza y procesa los datos.

Los sparse autoencoders respetan la estructura del "autoencoder simple" y simplemente se añade una **función de penalización** que fomenta activaciones promedio bajas en la capa latente, lo que provoca la dispersión. 
Además, es importante considerar que el **espacio latente** puede ser distinto al de los autoencoders simples. El espacio latente puede llegar a ser mayor que el tamaño de la entrada, pero esto dependerá del tipo de datos con los que se esté trabajando. Si los datos tienen una estructura común o baja complejidad, entonces requerirán un espacio latente más pequeño que la entrada. Por otro lado, si los datos son más complejos o variados, requerirán un espacio latente más grande.
Aunque en la mayoría de los SAEs usados en interpretabilidad de LLMs (Large Language Models) el tamaño del espacio latente suele ser mayor que el de la entrada. 

 **Nota: anotar el caso de la dimensionalidad para el ejemplo a abordar.** 

<details>
<summary> Nota: Información extra sobre las función de ponalización </summary>

Las funciones de penalización que inducen dispersión tienen en común que su objetivo es restringir el soporte efectivo del vector de representación latente, es decir, promover que la mayoría de sus componentes sean iguales o cercanos a cero. Para lograr esto, se introduce un término adicional en la función de pérdida que aumenta su valor cuando el número o magnitud de componentes diferentes de cero en la representación latente crece.

La condición de diferenciabilidad que suelen tener las funciones de penalización utilizadas en aprendizaje automático no es una necesidad teórica, sino una conveniencia computacional. En teoría, se pueden utilizar funciones no diferenciables como la norma L₀ exacta para inducir dispersión, ya que definen perfectamente bien el problema de optimización. Sin embargo, en la práctica, los métodos estándar como el descenso de gradiente requieren calcular derivadas, por lo que se favorecen penalizaciones suaves y diferenciables que permitan aplicar este tipo de algoritmos de forma eficiente.Por lo tanto, es totalmente válido prescindir de esta restricción si se está dispuesto a trabajar con enfoques diferentes, aunque estos podrían no ser óptimos desde el punto de vista computacional.

</details>

### **Funciones de Penalización**


**Divergencia KL**

   Se define $\rho$ como la activación deseada. La desviación de $\hat{\rho}_j$ respecto a $\rho$ se mide con la **Divergencia KL**:

   $$
   \mathrm{KL}\bigl(\rho \,\|\, \hat{\rho}_j\bigr)
   =
   \rho \,\log \frac{\rho}{\hat{\rho}_j}
   \;+\;
   (1-\rho)\,\log \frac{1-\rho}{\,1-\hat{\rho}_j}.
   $$

   Donde la activación promedio de la neurona $j$ es:

   $$
   \hat{\rho}_j = \frac{1}{N}\sum_{i=1}^N z_{i,j}.
   $$


<details>

<summary> Nota: Sobre la divergencia KL </summary>


**Definición General de la Divergencia KL**

La divergencia KL entre dos distribuciones de probabilidad $P(x)$ y $Q(x)$ se define como:

$$
D_{KL}(P \,\|\, Q) \;=\; \sum_{x} P(x)\,\log\!\Bigl(\tfrac{P(x)}{Q(x)}\Bigr).
$$

Donde, $P(x)$ es la distribución de referencia, $Q(x)$ es la distribución que usamos para aproximar a $P(x)$, La divergencia KL mide cuánta información se pierde cuando usamos $Q(x)$ en lugar de $P(x)$.

El principal objetivo de la divergencia de Kullback-Leibler (KL) es medir cuánta información se pierde cuando usamos una distribución de probabilidad 𝑄 para aproximar otra distribución P. En otras palabras, mide la diferencia entre dos distribuciones de probabilidad y nos dice cuánto nos alejamos de la distribución "verdadera" al usar una aproximación.


-- Imagen del articulo pendiente:  Solomon Kullback y Richard A. Leibler en su artículo de 1951: "On Information and Sufficiency".


**Divergencia KL con Bernoulli**

Cuando $P$ y $Q$ son distribuciones de Bernoulli con parámetros $\rho$ y $\hat{\rho}_j$, respectivamente, la variable aleatoria $X$ solo puede tomar los valores $0$ o $1$. Entonces:

- Para $X = 1$:
  
  $$
  P(1) = \rho, \quad Q(1) = \hat{\rho}_j.
  $$
  
- Para $X = 0$:
  
  $$
  P(0) = 1 - \rho, \quad Q(0) = 1 - \hat{\rho}_j.
  $$

Aplicamos la definición de la divergencia KL:

$$
D_{KL}(P \,\|\, Q) 
= \sum_{x \in \{0,1\}} P(x) \log \frac{P(x)}{Q(x)}.
$$

entonces,

$$
D_{KL}(\text{Bern}(\rho) \,\|\, \text{Bern}(\hat{\rho}_j))
=
\rho \,\log\!\Bigl(\tfrac{\rho}{\hat{\rho}_j}\Bigr)
\;+\;
(1-\rho)\,\log\!\Bigl(\tfrac{1-\rho}{1-\hat{\rho}_j}\Bigr).
$$

Esto nos da la divergencia KL específica para dos distribuciones Bernoulli.



</details>

**Penalización $L_0$**

Otra funcion de penalizacion en la que nos centraremos por su uso practico en la aplicación de este proyecto es la  regulacion **$L_0$**. La penalización $L_0$ (a veces denominada “norma $L_0$”) se define como el número de elementos distintos de cero en un vector.  
Si $z_i \in \mathbb{R}^m$ es el vector de activaciones de la capa oculta para la muestra $i$, entonces la “norma” $L_0$ de $z_i$ se expresa como:

$$
\| z_i \|_0 \;=\; \bigl|\{\,j : z_{i,j} \neq 0\,\}\bigr|.
$$

En otras palabras, $\| z_i \|_0$ es simplemente la cantidad de neuronas que están encendidas (activas) en la muestra $i$.  
Para todo el conjunto de datos, con $N$ muestras, la penalización $L_0$ se puede escribir de forma compacta como:

$$
L_0 \;=\; \sum_{i=1}^{N} \| z_i \|_0,
$$

o, de forma más explícita:

$$
L_0 \;=\; \sum_{i=1}^{N} \sum_{j=1}^{m} \mathbf{1}(z_{i,j} \neq 0),
$$

vale 1 si la condición se cumple y 0 en caso contrario, $z_{i,j}$ es la activación de la neurona $j$ en la muestra $i$, $N$ es el número de muestras de entrenamiento y $m$ es el número de neuronas en la capa oculta.
En la práctica, $L_0$ no es derivable con respecto a los parámetros del modelo, por lo que se suele aproximar o sustituir por otras penalizaciones (como $L_1$), que permiten métodos de optimización basados en gradientes.


<details>
<summary> Nota: Otras funciones de penalizacion comunes </summary>


Otra opción muy usada es la **regularización L1 sobre las activaciones**, que consiste en sumar el valor absoluto de todas las activaciones de la capa oculta:

$$
\text{L1} 
= \sum_{i=1}^{N} \sum_{j=1}^{m} \bigl|\,z_{i,j}\bigr|.
$$

Esta penalización empuja directamente las activaciones a ser lo más cercanas posible a cero, lo cual naturalmente genera *dispersión*.


También existe la **regularización L2 sobre activaciones**, que en lugar de tomar el valor absoluto, eleva las activaciones al cuadrado:

$$
\text{L2} 
= \sum_{i=1}^{N} \sum_{j=1}^{m} \bigl(z_{i,j}\bigr)^2.
$$

Esta penalización no genera dispersión tan fuerte como L1, pero ayuda a mantener las activaciones controladas.

</details>


### **Implementacion de la función de penalización**

La estructura general de la fución de perdida es la siguiente:
$$
   \mathcal{L}_{Sparse}
   =
   \mathcal{L}_{MSE}
   \;+\;
   \text{Función de penalización}$$

Basicamente se añade la funcion de penalizacion a la función de perdida.


**Ejemplo de como implementar**  
   El término de dispersión se agrega al MSE multiplicado por un factor $\beta$:

   $$
   \mathcal{L}_{Sparse}
   =
   \mathcal{L}_{MSE}
   \;+\;
   \beta \sum_{j=1}^{m} 
   \mathrm{KL}\bigl(\rho \,\|\, \hat{\rho}_j\bigr).
   $$

Este término adicional obliga a que la **activación promedio** de cada neurona $\hat{\rho}_j$ se acerque a $\rho$, convirtiendo así un autoencoder normal en un **autoencoder *disperso***.




##  **JumpReLU Sparse Autoencoder**

El JumpReLU Sparse Autoencoder es una arquitectura diseñada para obtener representaciones dispersas e interpretables de las activaciones internas de modelos de lenguaje grandes, como los transformers. Su propósito es decomponer vectores de activación en combinaciones lineales de un conjunto de direcciones base (features), de manera que solo unas pocas de estas se activen en cada caso, permitiendo análisis más interpretables y eficientes.
JumpReLU SAE emplea una activación modificada llamada JumpReLU, que introduce un umbral personalizado para cada feature. Esta pequeña variación permite mejorar tanto la dispersión como la fidelidad de reconstrucción sin sacrificar eficiencia computacional.
La importancia esta estructura de Autoencoder para este proyecto es que es la que se utilizara para los experimentos y sobre el modelo Gemma 2 (modelo Trasformer).


### **Estructura del modelo**

La estructura del modelo JumpReLU Sparse Autoencoder difiere poco del modelo de los Autoencoders Dispersos, y por lo tanto, también de los Autoencoders simples. Hay que notar la similitud entre este modelo y los demás. De esta forma, podemos observar que la diferencia principal está en el tipo de **función de activación del encoder**, su particular **función de penalización** que se integra, y los desafíos que implica incorporar dicha función de penalización. Igualmente, en este caso no se implementará una función de activación en la parte del decoder.


### **Codificador**

Toma como entrada una activación $x \in \mathbb{R}^n$ (proveniente de un transformer) y la proyecta a una representación dispersa $f(x) \in \mathbb{R}^m$, con $m > n$:

$$
\mathbf{z} = f_{\theta}(\mathbf{x}) = \sigma\bigl(W_e \,\mathbf{x} + \mathbf{b}_e\bigr) = \text{JumpReLU}_\theta(W_{\text{e}} x + b_{\text{e}})
$$

$\text{JumpReLU}_\theta$: función de activación descrita más abajo

### **Decodificador**

Reconstruye la activación original $x$ a partir de su representación dispersa:

$$
\hat{x} = W_{\text{d}} f(x) + b_{\text{d}}
$$

$W_{\text{d}} \in \mathbb{R}^{n \times M}$: matriz de pesos del decoder y $b_{\text{d}} \in \mathbb{R}^n$: vector de sesgos.

La razón por la que se decide evitar el uso de una función de activación en el decodificador es principalmente porque, en la aplicación de este autoencoder a nuestro modelo de lenguaje, la entrada y la salida del autoencoder son vectores reales, ya que representan activaciones internas del modelo. Estas activaciones pueden ser positivas o negativas, y el objetivo del SAE es reconstruir exactamente esas activaciones internas, no convertirlas en otra cosa.

### **Función de activación JumpReLU**

La función **JumpReLU** actúa como un ReLU con un umbral desplazado hacia la derecha. Se define como:

$$
\text{JumpReLU}_\theta(z_i) = 
\begin{cases}
z_i & \text{si } z_i > \theta_i \\
0 & \text{en otro caso}
\end{cases}
$$

O de forma compacta:

$$
\text{JumpReLU}_\theta(z) = z \cdot H(z - \theta)
$$

donde $H(\cdot)$ es la función escalón de Heaviside:

$$
H(a) =
\begin{cases}
1 & \text{si } a > 0 \\
0 & \text{si } a \leq 0
\end{cases}
$$

Esta función permite activar solo aquellos razgos cuya preactivación supere cierto umbral, evitando así "falsas activaciones" y mejorando la selectividad del encoder.


### **Función de pérdida**

El entrenamiento del JumpReLU SAE se realiza minimizando una función de pérdida.
La función de pérdida es:

$$
L(x) = \underbrace{\|x - \hat{x}(f(x))\|_2^2}_{\text{Error de reconstrucción}} + \lambda \cdot \underbrace{\|f(x)\|_0}_{\text{Penalización de dispersión (L0)}}
$$

Esta parte ya está más desallada en el apartado de las funciones de perdida. 


### **Entrenamiento con funciones no diferenciables**

La penalización $L_0$ y la función escalón $H(z - \theta)$ **no son diferenciables**, lo que complica el uso de backpropagation estándar. Para solucionarlo, se usa una técnica llamada **Straight-Through Estimator (STE)**, que permite calcular **gradientes aproximados** y así entrenar el modelo usando métodos convencionales de optimización.
